In [1]:
import math

def safe_log(x):
    return -math.inf if x == 0 else math.log(x)

def compute_log_prob_given_path(given_path: str, observed_seq: str) -> float:
    if len(given_path) != len(observed_seq):
        raise ValueError("Path and sequence are of different lengths.")

    log_prob = 0.0
    for i in range(len(observed_seq)):
        curr_state = given_path[i]
        obs = observed_seq[i]
        if i == 0:
            log_prob += safe_log(start_probs[curr_state])
        else:
            prev_state = given_path[i - 1]
            log_prob += safe_log(trans_probs[prev_state].get(curr_state, 0))
        log_prob += safe_log(emit_probs[curr_state].get(obs, 0))

    # Transition to End state (only from I)
    if given_path[-1] == 'I':
        log_prob += safe_log(trans_probs['I'].get('end', 0))

    return log_prob

# HMM parameters from Nature Primer
states = ['E', '5', 'I']
start_probs = {'E': 1.0, '5': 0.0, 'I': 0.0}

trans_probs = {
    'E': {'E': 0.9, '5': 0.1},
    '5': {'I': 1.0},
    'I': {'I': 0.9, 'end': 0.1},
}

emit_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}
}

# Inputs
manual_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"

# Log prob for given path
manual_log_prob = round(compute_log_prob_given_path(manual_path, observed_sequence), 2)
print("Log probability of given path:", manual_log_prob)

# Viterbi algorithm
viterbi = [{}]
viterbi_path = {}

# Initialization
for state in states:
    viterbi[0][state] = safe_log(start_probs[state]) + safe_log(emit_probs[state].get(observed_sequence[0], 0))
    viterbi_path[state] = [state]

# Recursion
for i in range(1, len(observed_sequence)):
    viterbi.append({})
    new_viterbi_path = {}

    for curr_state in states:
        max_logp = -math.inf
        best_prev_state = None

        for prev_state in states:
            trans_p = trans_probs.get(prev_state, {}).get(curr_state, 0)
            emit_p = emit_probs[curr_state].get(observed_sequence[i], 0)

            if trans_p > 0 and emit_p > 0:
                prob = viterbi[i-1][prev_state] + safe_log(trans_p) + safe_log(emit_p)
                if prob > max_logp:
                    max_logp = prob
                    best_prev_state = prev_state

        viterbi[i][curr_state] = max_logp
        if best_prev_state is not None:
            new_viterbi_path[curr_state] = viterbi_path[best_prev_state] + [curr_state]
        else:
            new_viterbi_path[curr_state] = [curr_state]

    viterbi_path = new_viterbi_path

# Termination
final_index = len(observed_sequence) - 1
final_log_prob, final_state = max((viterbi[final_index][s], s) for s in states)

print("Viterbi best log probability:", round(final_log_prob, 2))
print("Most likely path:", ''.join(viterbi_path[final_state]))


Log probability of given path: -41.22
Viterbi best log probability: -38.68
Most likely path: EEEEEEEEEEEEEEEEEEEEEEEEEE
